<a href="https://colab.research.google.com/github/wesley-wms/sistema-de-recomendacao-por-imagens-digitais/blob/main/SistemaRecomendacao.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --upgrade kagglehub

In [ ]:
import kagglehub
import os
import zipfile
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
from glob import glob

In [ ]:
dataset_path = "/root/.cache/kagglehub/datasets/bhaskar2443053/fashion-small/versions/1"
print("Arquivos e pastas no diretório do dataset:")
print(os.listdir(dataset_path))


image_dir = dataset_path  # Ajuste conforme necessário
data_paths = glob.glob(os.path.join(image_dir, "**/*.jpg"), recursive=True)

print(f"Total de imagens encontradas: {len(data_paths)}")
print("Exemplo de caminho:", data_paths[:5])  # Exibir algumas imagens encontradas


In [ ]:
# Carregar modelo pré-treinado para extração de features
model = ResNet50(weights='imagenet', include_top=False, pooling='avg')

In [ ]:
def extract_features(img_path, model):
    """Extrai características da imagem usando ResNet50."""
    img = load_img(img_path, target_size=(224, 224))
    img_array = img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = preprocess_input(img_array)
    features = model.predict(img_array)
    return features.flatten()

# Extrair características para todas as imagens
df_features = pd.DataFrame()
df_features['image_path'] = data_paths
df_features['features'] = df_features['image_path'].apply(lambda x: extract_features(x, model))

# Criar matriz de características
feature_matrix = np.vstack(df_features['features'].values)

In [ ]:
# Função para recomendar imagens similares
def recommend_images(query_img_path, model, feature_matrix, df_features, top_n=5):
    query_features = extract_features(query_img_path, model).reshape(1, -1)
    similarities = cosine_similarity(query_features, feature_matrix)[0]
    indices = np.argsort(similarities)[::-1][:top_n]
    recommended_images = df_features.iloc[indices]['image_path'].values
    return recommended_images

query_image = data_paths[0]  # Escolher uma imagem do dataset
recommended_images = recommend_images(query_image, model, feature_matrix, df_features)

In [ ]:
# Exibir a imagem consultada e as recomendações
plt.figure(figsize=(10, 5))
plt.subplot(1, top_n + 1, 1)
plt.imshow(load_img(query_image))
plt.title("Imagem Consultada")
plt.axis("off")

for i, img_path in enumerate(recommended_images):
    plt.subplot(1, top_n + 1, i + 2)
    plt.imshow(load_img(img_path))
    plt.title(f"Recomendação {i+1}")
    plt.axis("off")

plt.show()